In [1]:
import sklearn_crfsuite
from sklearn_crfsuite.metrics import flat_f1_score
from sklearn_crfsuite import CRF
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
from spacy.lang.en import English
from spacy import displacy
from pathlib import Path
import joblib
from spacy.tokens import Doc
from typing import List
import random
from statistics import mean, stdev
from loguru import logger
import sys
import spacy
from medspacy_io.reader.brat_reader import BratDocReader, BratDirReader
import medspacy

## Initial Brat Reader

In [2]:
cleaned_train_dir=r'..\data\n2c2\cleaned_training'
cleaned_test_dir=r'..\data\n2c2\cleaned_test'
Path(cleaned_train_dir).exists(), Path(cleaned_test_dir).exists()

(True, True)

In [3]:
nlp=spacy.load('en_core_web_sm', disable=['ner'])

In [4]:
dir_reader = BratDirReader(nlp=nlp, schema_file=str(Path(cleaned_train_dir, 'annotation.conf')), support_overlap=True)

## read ehost annotation/load pickle

In [5]:
pickle_file= r'..\data\n2c2\spacy_docs.joblib'

In [6]:
if not Path(pickle_file).exists():
    train_docs=dir_reader.read(txt_dir=cleaned_train_dir)
    test_docs=dir_reader.read(txt_dir=cleaned_test_dir)
    print(len(train_docs), len(test_docs))
    joblib.dump((train_docs, test_docs), pickle_file)
else:
    print(f'{pickle_file} already exists, load them directly')
    # before load from pickle, initiate EhostDirReader or EhostDocReader first, because some Doc extension used to store meta data will not be automatically recreated by loading.
    train_docs, test_docs=joblib.load(pickle_file)

..\data\n2c2\spacy_docs.joblib already exists, load them directly


## define sampling functions

In [7]:
rounds =10
seed= 14

In [8]:
len(train_docs), len(test_docs)

(303, 202)

## Define CRF Wrapper

In [9]:
from CRFWrapper_Sentence import spans_to_bio, convert_docs, word2features, sent2features,compute_metrics_and_averages,  CRFModel


In [10]:
## Get all annotation types: 
annos=set()
for d in train_docs:
    for anno in d.spans.keys():
        annos.add(anno)
print(annos)

{'Form', 'Frequency', 'Duration', 'Route', 'Dosage', 'Drug', 'Strength', 'Reason', 'ADE'}


In [11]:
crf_model=CRFModel(anno_types=annos)

## Convert training and testing docs into sentence level dataframe

In [12]:
_, train_df=convert_docs(train_docs, anno_types=annos)
_, test_df=convert_docs(test_docs, anno_types=annos)

In [13]:
train_df

,sentence_id,doc_name,token,label
0,0,100035.txt,Admission,O
1,0,100035.txt,Date,O
2,0,100035.txt,:,O
3,1,100035.txt,[,O
4,1,100035.txt,*,O
...,...,...,...,...
3151,929927,198406.txt,Followup,O
3152,929927,198406.txt,Instructions,O
3153,929927,198406.txt,:,O
3154,929927,198406.txt,\n,O


In [14]:
train_sentID_list = train_df['sentence_id'].to_list()
train_sentID_set = set(train_sentID_list)
train_sentID_uniqList = list(train_sentID_set)
test_sentID_list = test_df['sentence_id'].to_list()
test_sentID_set = set(test_sentID_list)
test_sentID_uniqList = list(test_sentID_set)
print(len(train_sentID_uniqList), len(test_sentID_uniqList))

51798 34334


In [15]:
int(1.0*len(train_sentID_uniqList)/10)

5179

# Random Sampling Simulator

In [16]:
from ALLSampler_Sentence import SamplingSimulator, ModelSamplingSimulator, RandomSamplingSimulator

In [17]:
r_simulator=RandomSamplingSimulator(total_sents=train_df, 
                                    total_round=10, 
                                    modelWrapper=crf_model, 
                                    eval_sents=test_df, 
                                    init_seed=seed)

## test run

In [18]:
scores=r_simulator.simulate_rounds(boostrap_times=200) #sample sentences for 10 rounds; each round bootstrap sampling 200 for evaluation

2024-04-24 12:40:38.789 | INFO     | ALLSampler_Sentence:simulate_rounds:92 - simulate round 0.
2024-04-24 12:40:38.790 | DEBUG    | ALLSampler_Sentence:keep_sample:75 - The first round sampling will be random
2024-04-24 12:40:39.208 | INFO     | ALLSampler_Sentence:keep_sample:82 - current sampled sentences: 5179, remaining sentences: 46619
2024-04-24 12:40:41.315 | DEBUG    | CRFWrapper_Sentence:fit:284 - Reset and train CRF model...
2024-04-24 12:41:43.962 | DEBUG    | CRFWrapper_Sentence:fit:292 - Training complete.
2024-04-24 12:41:44.052 | DEBUG    | CRFWrapper_Sentence:bootstrap_eval_DFsent:352 - Predicting eval sents...
2024-04-24 12:42:10.450 | DEBUG    | CRFWrapper_Sentence:bootstrap_eval_DFsent:356 - Calculate scores from bootstrapping 200 times
2024-04-24 12:46:36.887 | DEBUG    | CRFWrapper_Sentence:bootstrap_eval_DFsent:365 - complete
2024-04-24 12:46:37.017 | DEBUG    | ALLSampler_Sentence:simulate_rounds:103 - {'precision': 0.9460863337731906, 'recall': 0.77182582947748

## Bootstrap 3 different initials runs

In [18]:
boostrap_runs=3
total_round=10
logger.remove()
logger.add(sys.stderr, level='INFO')

1

In [19]:
random.seed(14)
seeds=[random.randint(1,10000000) for  _ in range(boostrap_runs)]
seeds

[1792286, 8843471, 4142887]

In [20]:
all_scores=[]
for si, seed  in enumerate(seeds):
    logger.info(f'start run {si}.')
    crf_model=CRFModel(anno_types=annos)
    r_simulator=RandomSamplingSimulator(total_sents=train_df, 
                                    total_round=10, 
                                    modelWrapper=crf_model, 
                                    eval_sents=test_df, 
                                    init_seed=seed)  
    scores=r_simulator.simulate_rounds()
    all_scores.append(scores) 

2024-05-03 23:07:15.669 | INFO     | __main__:<module>:3 - start run 0.
2024-05-03 23:07:15.781 | INFO     | ALLSampler_Sentence:simulate_rounds:92 - simulate round 0.
2024-05-03 23:07:16.160 | INFO     | ALLSampler_Sentence:keep_sample:82 - current sampled sentences: 2589, remaining sentences: 49209
2024-05-03 23:12:21.298 | INFO     | ALLSampler_Sentence:simulate_rounds:92 - simulate round 1.
2024-05-03 23:12:21.677 | INFO     | ALLSampler_Sentence:keep_sample:82 - current sampled sentences: 5178, remaining sentences: 46620
2024-05-03 23:17:58.302 | INFO     | ALLSampler_Sentence:simulate_rounds:92 - simulate round 2.
2024-05-03 23:17:58.648 | INFO     | ALLSampler_Sentence:keep_sample:82 - current sampled sentences: 7767, remaining sentences: 44031
2024-05-03 23:24:03.225 | INFO     | ALLSampler_Sentence:simulate_rounds:92 - simulate round 3.
2024-05-03 23:24:03.565 | INFO     | ALLSampler_Sentence:keep_sample:82 - current sampled sentences: 10356, remaining sentences: 41442
2024-05

In [22]:
joblib.dump(all_scores, r'../data/n2c2/scores_sentence_sampling/ner_random_scores_sentenceSampling.joblib')


['../data/n2c2/scores_sentence_sampling/ner_random_scores_sentenceSampling.joblib']